In [1]:
import black
import fasttext
import jupyter_black
import json
import keras
import matplotlib.pyplot as plt
import nltk
import numpy as np
import os
import pandas as pd
import seaborn as sns
import tensorflow as tf
import random
from collections import Counter
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import (
    Bidirectional,
    Dense,
    Dropout,
    Embedding,
    LSTM,
    GRU
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score
from sklearn.model_selection import (
    cross_val_score,
    train_test_split,
    StratifiedKFold,
)
import xml.etree.ElementTree as ET

# Setting options
pd.options.mode.chained_assignment = None  # default='warn'
nltk.download("stopwords")
nltk.download("punkt")
#black for linting reasons
jupyter_black.load(
    lab=False,
    line_length=80,
    verbosity="DEBUG",
    target_version=black.TargetVersion.PY310,
)
pd.set_option('display.max_colwidth', None)
print("All good!")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Cacu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Cacu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
DEBUG:jupyter_black:config: {'line_length': 80, 'target_versions': {<TargetVersion.PY310: 10>}}


<IPython.core.display.Javascript object>

All good!


In [ ]:
csv_file_path = r"C:\Users\Cacu\Desktop\Universidad\Trabajo_Final\DataSets\open-dataset-for-sentiment-analysis-master\betsentiment-ES-tweets-sentiment-worldcup.csv"


# Function to extract sentiment scores from the json str
def extract_sentiment_scores(json_str):
    sentiment_data = json.loads(json_str)
    return (
        sentiment_data["Neutral"],
        sentiment_data["Negative"],
        sentiment_data["Positive"],
        sentiment_data["Mixed"],
    )


# Define data types
dtype_dict = {
    "tweet_date_created": str,
    "tweet_id": int,
    "tweet_text": str,
    "language": str,
    "sentiment": str,
}

try:
    df = pd.read_csv(csv_file_path, encoding="utf-8", dtype=dtype_dict)
except UnicodeDecodeError:
    df = pd.read_csv(csv_file_path, encoding="latin-1", dtype=dtype_dict)

sentiment_scores_list = df["sentiment_score"].map(extract_sentiment_scores)

sentiment_scores_df = pd.DataFrame(
    sentiment_scores_list.tolist(),
    columns=["Neutral", "Negative", "Positive", "Mixed"],
)

df = pd.concat([df, sentiment_scores_df], axis=1)

df.drop("sentiment_score", axis=1, inplace=True)

In [ ]:
df

In [ ]:
# Read the stopwords from the .txt file and store them in a list
custom_stopwords_file = (
    r"C:\Users\Cacu\Desktop\Universidad\Trabajo_Final\utilities\stopwords.txt"
)
with open(custom_stopwords_file, "r", encoding="utf-8") as file:
    custom_stopwords_list = [line.strip() for line in file]

# Create an empty set to hold the stopwords
custom_stopwords_set = set()

# Add the stopwords from the list to the set
custom_stopwords_set.update(custom_stopwords_list)

In [ ]:
# Specify the Snowball stemmer for Spanish
stemmer = SnowballStemmer("spanish")

# 01 - Convert NaN values to an empty string
df["tweet_text"] = df["tweet_text"].fillna("")

# 02 - Lowercasing
df["tweet_text"] = df["tweet_text"].str.lower()

# 03 - Removing punctuation
df["tweet_text"] = df["tweet_text"].str.replace("[^\w\s]", "")

# 04 - Tokenization
df["tokens"] = df["tweet_text"].apply(word_tokenize)

# 05 - Removing stop words
stop_words = set(stopwords.words("spanish"))
df["filtered_tokens"] = df["tokens"].apply(
    lambda tokens: [word for word in tokens if word not in stop_words]
)

# 06 - Stemming
df["stemmed_tokens"] = df["filtered_tokens"].apply(
    lambda tokens: [stemmer.stem(word) for word in tokens]
)

In [ ]:
df

## 1. Embedding

fastText es una biblioteca para el aprendizaje de incrustaciones de palabras y clasificación de texto creada por el laboratorio de Investigación de Inteligencia Artificial de Facebook (FAIR). El modelo es un algoritmo de aprendizaje no supervisado para obtener representaciones vectoriales de palabras. Facebook pone a disposición modelos preentrenados para 294 idiomas. fastText utiliza una red neuronal para la incrustación de palabras [Fuente: Wikipedia].

Documentación sobre Gensim: models.fasttext

FastText es una extensión de Word2Vec propuesta por Facebook en 2016. En lugar de alimentar palabras individuales en la Red Neuronal, FastText divide las palabras en varios n-gramas (subpalabras). Por ejemplo, los trigramas para la palabra "manzana" son "man," "anz," y "nza" (ignorando los límites de inicio y fin de las palabras). El vector de incrustación de la palabra "manzana" será la suma de todos estos n-gramas. Después de entrenar la Red Neuronal, tendremos incrustaciones de palabras para todos los n-gramas dados el conjunto de datos de entrenamiento. Las palabras raras ahora pueden representarse adecuadamente, ya que es muy probable que algunos de sus n-gramas también aparezcan en otras palabras. Te mostraré cómo usar FastText con Gensim en la siguiente sección.

In [ ]:
df["tokens_as_string"] = df["tokens"].apply(" ".join)
# Save the stemmed tokens to a text file (one sentence per line)
with open("tokens.txt", "w") as f:
    f.write("\n".join(df["tokens_as_string"]))

In [ ]:
# Fasttext model on stemmed tokens
model = fasttext.train_unsupervised("tokens.txt", model="skipgram")

# Create tweet-level embeddings using the trained model
embeddings = []

In [ ]:
for tokens in df["tokens_as_string"]:
    vector = model.get_sentence_vector(tokens)
    embeddings.append(vector)

# Convert the embeddings to a DataFrame
embedding_df = pd.DataFrame(embeddings)

In [ ]:
# embeddings df with the original dataset
df_worldcup_embeddings = pd.concat([df, embedding_df], axis=1)

# embedding df to csv
df_worldcup_embeddings.to_csv("2018_dataset_with_embeddings.csv", index=False)

model.save_model("model_worldcup_embedding.bin")

In [ ]:
df_worldcup_embeddings

In [82]:
nrows_to_load = 350000  # Adjust this to the desired subset size

# start from here - csv already generated
df = pd.read_csv("2018_dataset_with_embeddings.csv", nrows=nrows_to_load)
# shuffle the DataFrame rows
df = df.sample(frac=1)

## Preparing for training

In [83]:
df.count()

tweet_date_created    350000
tweet_id              350000
tweet_text            350000
language              350000
sentiment             350000
                       ...  
95                    350000
96                    350000
97                    350000
98                    350000
99                    350000
Length: 113, dtype: int64

In [85]:
label_mapping = {"POSITIVE": 2, "NEUTRAL": 1, "NEGATIVE": 0}
df["sentiment_encoded"] = df["sentiment"].map(label_mapping)

In [87]:
print(df.isnull().sum())

# Remove rows with missing values
df.dropna(inplace=True)

# Remove duplicates
df.drop_duplicates(subset=["tweet_text"], keep="first", inplace=True)

tweet_date_created       0
tweet_id                 0
tweet_text               0
language                 0
sentiment                0
                      ... 
96                       0
97                       0
98                       0
99                       0
sentiment_encoded     2132
Length: 114, dtype: int64


In [88]:
df

,tweet_date_created,tweet_id,tweet_text,language,sentiment,Neutral,Negative,Positive,Mixed,tokens,filtered_tokens,stemmed_tokens,tokens_as_string,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,sentiment_encoded
60151,2018-06-28T16:12:59.384000,1784107008,másunidosquenunca conlafeintacta vamoscolombia conlatricolorpuesta mundialrusia2018 vamosaganar nonosvamosarendir gracias colombia gracias x vencer el socialismo xxi gracias farid mondragon y gracias por avanzar estamosmelospaoctavos\nmlagder pilica7 mhoyos333 httpstcov2uudmircq,es,POSITIVE,0.350060,0.000235,0.644539,0.005166,"['másunidosquenunca', 'conlafeintacta', 'vamoscolombia', 'conlatricolorpuesta', 'mundialrusia2018', 'vamosaganar', 'nonosvamosarendir', 'gracias', 'colombia', 'gracias', 'x', 'vencer', 'el', 'socialismo', 'xxi', 'gracias', 'farid', 'mondragon', 'y', 'gracias', 'por', 'avanzar', 'estamosmelospaoctavos', 'mlagder', 'pilica7', 'mhoyos333', 'httpstcov2uudmircq']","['másunidosquenunca', 'conlafeintacta', 'vamoscolombia', 'conlatricolorpuesta', 'mundialrusia2018', 'vamosaganar', 'nonosvamosarendir', 'gracias', 'colombia', 'gracias', 'x', 'vencer', 'socialismo', 'xxi', 'gracias', 'farid', 'mondragon', 'gracias', 'avanzar', 'estamosmelospaoctavos', 'mlagder', 'pilica7', 'mhoyos333', 'httpstcov2uudmircq']","['masunidosquenunc', 'conlafeintact', 'vamoscolombi', 'conlatricolorpuest', 'mundialrusia2018', 'vamosagan', 'nonosvamosarend', 'graci', 'colombi', 'graci', 'x', 'venc', 'social', 'xxi', 'graci', 'far', 'mondragon', 'graci', 'avanz', 'estamosmelospaoctav', 'mlagd', 'pilica7', 'mhoyos333', 'httpstcov2uudmircq']",másunidosquenunca conlafeintacta vamoscolombia conlatricolorpuesta mundialrusia2018 vamosaganar nonosvamosarendir gracias colombia gracias x vencer el socialismo xxi gracias farid mondragon y gracias por avanzar estamosmelospaoctavos mlagder pilica7 mhoyos333 httpstcov2uudmircq,-0.029059,0.089238,-0.099062,0.060049,0.159505,0.221087,0.059089,-0.063663,0.054963,0.018248,0.054835,-0.024155,-0.028379,-0.075933,0.009605,0.123610,-0.075535,0.003957,0.056147,0.054395,0.069691,0.078577,-0.029189,-0.030220,0.070053,-0.000857,0.061832,-0.003341,0.054546,0.127368,-0.057978,0.000042,-0.023296,-0.004476,-0.012103,0.025943,0.053024,0.035395,-0.064056,0.107801,0.056701,-0.022982,0.092009,-0.009205,0.050853,-0.086181,0.092590,-0.032310,-0.062916,-0.067546,-0.064600,-0.068724,-0.021663,0.103924,-0.016985,0.005648,-0.071782,0.060273,0.007180,0.029772,0.006658,0.047375,-0.005312,-0.008307,0.084346,0.022886,-0.063585,-0.019063,-0.054511,0.139538,-0.012181,0.016962,-0.060934,-0.048208,0.066808,0.038369,-0.007719,0.060613,0.009058,0.046057,-0.009863,0.079630,0.017453,-0.006552,0.037371,-0.099145,0.089840,0.027956,-0.054526,0.010925,-0.047625,0.024872,0.041494,-0.017940,-0.021979,-0.050140,0.054040,-0.049168,-0.055790,0.049390,2.0
70375,2018-06-22T12:18:11,-1017774079,para cuándo la fcfseleccioncol jugándole como costa rica a brasil,es,NEUTRAL,0.978532,0.006588,0.012492,0.002388,"['para', 'cuándo', 'la', 'fcfseleccioncol', 'jugándole', 'como', 'costa', 'rica', 'a', 'brasil']","['cuándo', 'fcfseleccioncol', 'jugándole', 'costa', 'rica', 'brasil']","['cuand', 'fcfseleccioncol', 'jug', 'cost', 'ric', 'brasil']",para cuándo la fcfseleccioncol jugándole como costa rica a brasil,-0.025459,0.043306,-0.114674,0.107244,0.160624,0.195959,0.105956,-0.005502,0.058612,0.012669,0.129113,-0.093427,0.019399,-0.099694,0.053587,0.064284,-0.040992,-0.008989,0.063830,0.034736,0.054009,0.078707,-0.055019,0.002308,0.030089,0.043087,0.013144,-0.019816,0.046936,0.094541,-0.015879,-0.136571,0.051188,0.021494,-0.151703,0.084773,0.096507,-0.050720,-0.065546,0.039195,0.113884,-0.014540,0.049530,0.076624,0.025680,-0.085711,0.062923,0.006464,0.020467,-0.044417,-0.01

In [62]:
vocab_size = 282459  # number of distinct tokens
dimensions = 100  # dimensions of embedding fasttext
max_sequence_length = 35  # number of tokens per tweet

In [89]:
# Set the max_columns option to None to display all columns
pd.set_option("display.max_columns", None)

df

,tweet_date_created,tweet_id,tweet_text,language,sentiment,Neutral,Negative,Positive,Mixed,tokens,filtered_tokens,stemmed_tokens,tokens_as_string,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,sentiment_encoded
60151,2018-06-28T16:12:59.384000,1784107008,másunidosquenunca conlafeintacta vamoscolombia conlatricolorpuesta mundialrusia2018 vamosaganar nonosvamosarendir gracias colombia gracias x vencer el socialismo xxi gracias farid mondragon y gracias por avanzar estamosmelospaoctavos\nmlagder pilica7 mhoyos333 httpstcov2uudmircq,es,POSITIVE,0.350060,0.000235,0.644539,0.005166,"['másunidosquenunca', 'conlafeintacta', 'vamoscolombia', 'conlatricolorpuesta', 'mundialrusia2018', 'vamosaganar', 'nonosvamosarendir', 'gracias', 'colombia', 'gracias', 'x', 'vencer', 'el', 'socialismo', 'xxi', 'gracias', 'farid', 'mondragon', 'y', 'gracias', 'por', 'avanzar', 'estamosmelospaoctavos', 'mlagder', 'pilica7', 'mhoyos333', 'httpstcov2uudmircq']","['másunidosquenunca', 'conlafeintacta', 'vamoscolombia', 'conlatricolorpuesta', 'mundialrusia2018', 'vamosaganar', 'nonosvamosarendir', 'gracias', 'colombia', 'gracias', 'x', 'vencer', 'socialismo', 'xxi', 'gracias', 'farid', 'mondragon', 'gracias', 'avanzar', 'estamosmelospaoctavos', 'mlagder', 'pilica7', 'mhoyos333', 'httpstcov2uudmircq']","['masunidosquenunc', 'conlafeintact', 'vamoscolombi', 'conlatricolorpuest', 'mundialrusia2018', 'vamosagan', 'nonosvamosarend', 'graci', 'colombi', 'graci', 'x', 'venc', 'social', 'xxi', 'graci', 'far', 'mondragon', 'graci', 'avanz', 'estamosmelospaoctav', 'mlagd', 'pilica7', 'mhoyos333', 'httpstcov2uudmircq']",másunidosquenunca conlafeintacta vamoscolombia conlatricolorpuesta mundialrusia2018 vamosaganar nonosvamosarendir gracias colombia gracias x vencer el socialismo xxi gracias farid mondragon y gracias por avanzar estamosmelospaoctavos mlagder pilica7 mhoyos333 httpstcov2uudmircq,-0.029059,0.089238,-0.099062,0.060049,0.159505,0.221087,0.059089,-0.063663,0.054963,0.018248,0.054835,-0.024155,-0.028379,-0.075933,0.009605,0.123610,-0.075535,0.003957,0.056147,0.054395,0.069691,0.078577,-0.029189,-0.030220,0.070053,-0.000857,0.061832,-0.003341,0.054546,0.127368,-0.057978,0.000042,-0.023296,-0.004476,-0.012103,0.025943,0.053024,0.035395,-0.064056,0.107801,0.056701,-0.022982,0.092009,-0.009205,0.050853,-0.086181,0.092590,-0.032310,-0.062916,-0.067546,-0.064600,-0.068724,-0.021663,0.103924,-0.016985,0.005648,-0.071782,0.060273,0.007180,0.029772,0.006658,0.047375,-0.005312,-0.008307,0.084346,0.022886,-0.063585,-0.019063,-0.054511,0.139538,-0.012181,0.016962,-0.060934,-0.048208,0.066808,0.038369,-0.007719,0.060613,0.009058,0.046057,-0.009863,0.079630,0.017453,-0.006552,0.037371,-0.099145,0.089840,0.027956,-0.054526,0.010925,-0.047625,0.024872,0.041494,-0.017940,-0.021979,-0.050140,0.054040,-0.049168,-0.055790,0.049390,2.0
70375,2018-06-22T12:18:11,-1017774079,para cuándo la fcfseleccioncol jugándole como costa rica a brasil,es,NEUTRAL,0.978532,0.006588,0.012492,0.002388,"['para', 'cuándo', 'la', 'fcfseleccioncol', 'jugándole', 'como', 'costa', 'rica', 'a', 'brasil']","['cuándo', 'fcfseleccioncol', 'jugándole', 'costa', 'rica', 'brasil']","['cuand', 'fcfseleccioncol', 'jug', 'cost', 'ric', 'brasil']",para cuándo la fcfseleccioncol jugándole como costa rica a brasil,-0.025459,0.043306,-0.114674,0.107244,0.160624,0.195959,0.105956,-0.005502,0.058612,0.012669,0.129113,-0.093427,0.019399,-0.099694,0.053587,0.064284,-0.040992,-0.008989,0.063830,0.034736,0.054009,0.078707,-0.055019,0.002308,0.030089,0.043087,0.013144,-0.019816,0.046936,0.094541,-0.015879,-0.136571,0.051188,0.021494,-0.151703,0.084773,0.096507,-0.050720,-0.065546,0.039195,0.113884,-0.014540,0.049530,0.076624,0.025680,-0.085711,0.062923,0.006464,0.020467,-0.044417,-0.01

In [90]:
# Extract X (features) and y (target)
X = df.iloc[:, 13:-1].to_numpy()
y = df["sentiment_encoded"].to_numpy()

# Make sure y is in the correct shape
y = np.reshape(y, (-1, 1))

In [91]:
y

array([[2.],
       [1.],
       [1.],
       ...,
       [1.],
       [0.],
       [0.]])

In [92]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [93]:
X_train

array([[-0.02098395,  0.08056071, -0.03259998, ..., -0.05141212,
        -0.00999504, -0.01927253],
       [-0.04503735,  0.07792307, -0.08982849, ...,  0.01504025,
        -0.04177724,  0.08012953],
       [ 0.03346503,  0.07148388, -0.06787509, ..., -0.02803733,
         0.00856456, -0.00141171],
       ...,
       [-0.02056255,  0.08412792, -0.14409721, ...,  0.00446333,
        -0.03497283,  0.05015467],
       [-0.01023732,  0.09164905, -0.05616085, ..., -0.01642906,
         0.0237581 , -0.00789321],
       [-0.00020143,  0.02998669,  0.02280423, ..., -0.06988493,
        -0.01309495,  0.06138528]])

In [94]:
y_train

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]])

In [95]:
print(y_train[:1000])  # Print the first 10 label values

[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [2.]
 [2.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [2.]
 [1.]
 [1.]
 [1.]
 [2.]
 [1.]
 [1.]
 [1.]
 [2.]
 [2.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [2.]
 [1.]
 [2.]
 [1.]
 [2.]
 [1.]
 [2.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [2.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [2.]
 [1.]
 [0.]
 [1.]
 [1.]
 [2.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [2.]
 [2.]
 [2.]
 [2.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [2.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [2.]
 [2.]
 [1.]
 [2.]
 [1.]
 [0.]
 [1.]
 [1.]
 [2.]
 [1.]
 [0.]
 [1.]
 [2.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [2.]
 [2.]
 [1.]
 [2.]
 [2.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [2.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [2.]
 [1.]
 [1.]
 [1.]
 [1.]
 [2.]
 [0.]
 [2.]
 [1.]
 [2.]
 [1.]
 [1.

In [96]:
# Initialize a Tokenizer
tokenizer = Tokenizer()
# Fit the tokenizer on your tokenized text data
tokenizer.fit_on_texts(df["tokens"])

# Get the vocabulary size
vocab_size = len(tokenizer.word_index) + 1  # Add 1 for the zero padding

# Print the vocabulary size
print("Vocabulary size (input_dim):", vocab_size)

Vocabulary size (input_dim): 318203


In [97]:
# Define the model
model = Sequential()
model.add(
    Embedding(
        input_dim=vocab_size,
        output_dim=50,
        input_length=100,
    )
)
model.add(LSTM(100))
model.add(Dense(3, activation="softmax"))

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Compile the model
model.compile(
    loss=SparseCategoricalCrossentropy(),
    optimizer=Adam(learning_rate=0.001),
    metrics=["accuracy"],
)

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

Epoch 1/10
7333/7333 [==============================] - 1068s 146ms/step - loss: 0.7524 - accuracy: 0.7401
Epoch 2/10
7333/7333 [==============================] - 1041s 142ms/step - loss: 0.7513 - accuracy: 0.7402
Epoch 3/10
7333/7333 [==============================] - 1044s 142ms/step - loss: 0.7511 - accuracy: 0.7402
Epoch 4/10
7333/7333 [==============================] - 1037s 141ms/step - loss: 0.7510 - accuracy: 0.7402
Epoch 5/10
3804/7333 [==============>...............] - ETA: 8:15 - loss: 0.7488 - accuracy: 0.7414